In [28]:
import tensorflow as tf
import numpy as np
from SimDataDB import SimDataDB
%matplotlib inline
from matplotlib import pylab as plt
from matplotlib.pyplot import imshow

In [29]:
sdb = SimDataDB('pix.db')

In [6]:
sess = tf.InteractiveSession()

In [84]:
r=np.vstack(sdb.Query('select pic from static order by random() limit 10'))

Failed to sort.


In [86]:
r.shape

(10, 32, 32, 3)

In [186]:
def gen():
    """Returns the 3 channel image"""
    while True:
        yield np.vstack(sdb.Query('select pic from static order by random() limit 10')),
data = tf.data.Dataset.from_generator(gen, (tf.float32,),(tf.TensorShape([None,32,32,3]),) )

In [187]:
def gen():
    """Returns the energy and the b&w fracture pic"""
    while True:
        r = sdb.Query('select W,pic from static order by random() limit 10')
        yield np.vstack([_[0] for _ in r]),1-np.stack([_[1][:,:,2] for _ in r]),
data = tf.data.Dataset.from_generator(gen, (tf.float32,tf.float32,),(tf.TensorShape([None,1]),tf.TensorShape([None,32,32,]),) )

In [177]:
#data = tf.data.experimental.SqlDataset("sqlite", 'pix.db',
#                                       'select W,G_c,pic from static',
#                                       (tf.float64,tf.float64,tf.Tensor))

In [178]:
iterator = data.make_one_shot_iterator()
nxt = iterator.get_next()

In [204]:
i_W = nxt[0]
i_pic = nxt[1]
A = tf.Variable(tf.truncated_normal(shape=(32,32,1),stddev=0.1))
o_W = tf.einsum('ijk,jkl->il',i_pic,A)
o_goal = tf.losses.mean_squared_error(o_W,i_W)

In [ ]:
o_goal = tf.reduce_mean

In [198]:
learning_rate = 1e-2
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(o_goal, var_list=[A])

In [208]:
init = tf.global_variables_initializer()

In [209]:
sess.run(init)
for i in range(200):
    sess.run(train_step)
    print sess.run(o_goal)

0.8595235
0.38722596
1.1400604
0.22551927
0.41321498
0.19796798
0.12507287
0.26432502
0.14211819
0.12416021
0.13793738
0.12571129
0.15398066
0.2439026
0.0677676
0.12264286
0.08811759
0.3124218
0.10302564
0.08818773
0.14578156
0.13803728
0.2343292
0.18365939
0.09059195
0.099130765
0.047309518
0.06912633
0.08366527
0.06740996
0.13703796
0.12950966
0.2394164
0.025162125
0.1289521
0.086825825
0.17121369
0.12892142
0.027387878
0.076898076
0.04990658
0.12562017
0.05295077
0.06322077
0.082472935
0.05753836
0.05790922
0.06838696
0.15059398
0.011606638
0.0715858
0.07732662
0.056374438
0.10884364
0.05596233
0.08264726
0.0637099
0.06597678
0.01915214
0.019128071
0.010777025
0.090542994
0.038089078
0.042621918
0.07491053
0.04297199
0.028612291
0.027192503
0.027108675
0.033115678
0.06880256
0.02467086
0.075858116
0.036387254
0.037585426
0.03796685
0.025122052
0.032316662
0.09778456
0.014286813
0.0072779926
0.19062862
0.023480158
0.012479847
0.064223714
0.0080105495
0.057836317
0.01878149
0.02917495

In [210]:
imshow(r[1][3,:,:])

IndexError: too many indices for array